# 07 - Reranking y Optimización de Retrieval

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2 horas

---

## Índice

1. [Introducción al Reranking](#intro)
2. [Cross-Encoder](#cross)
3. [Bi-Encoder (Dense Retrieval)](#bi)
4. [LLM como Reranker](#llm)
5. [Estrategias combinadas](#combinadas)
6. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Entender por qué el reranking mejora la calidad del RAG
- Implementar Cross-Encoder para reranking preciso
- Comparar Bi-Encoder vs Cross-Encoder
- Combinar múltiples estrategias de retrieval

<a name="intro"></a>
## 1. Introducción al Reranking

### ¿Por qué necesitamos Reranking?

La primera fase de retrieval (Bi-Encoder + similitud coseno) es **rápida pero imprecisa**. El reranking añade una segunda fase más precisa pero más costosa.

```
Query → [Bi-Encoder] → Top 50 candidatos → [Reranker] → Top 5 finales
         (rápido)                           (preciso)
```

### Tipos de Rerankers

| Tipo | Descripción | Precisión | Velocidad |
|------|-------------|-----------|----------|
| **Cross-Encoder** | Evalúa (query, doc) juntos | Alta | Lenta |
| **Bi-Encoder** | Embeddings separados | Media | Rápida |
| **LLM Reranker** | LLM evalúa relevancia | Muy alta | Muy lenta |

In [ ]:
# Install required libraries (all free)
!pip install -q sentence-transformers torch

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Sample documents for testing
query = "¿Cuáles son las aplicaciones más prometedoras de la IA en medicina?"

documents = [
    "La IA puede acelerar el descubrimiento de nuevos medicamentos para diversos tipos de cáncer.",
    "En cardiología, los modelos predictivos ayudan a identificar pacientes con alto riesgo de infarto.",
    "Los sistemas de IA permiten analizar imágenes médicas para detectar tumores en etapas tempranas.",
    "La IA ha demostrado ser útil en el monitoreo de enfermedades crónicas como la diabetes.",
    "La IA se utiliza para personalizar tratamientos oncológicos según el perfil genético.",
    "En neurología, se emplea IA para detectar patrones de deterioro cognitivo.",
    "Algoritmos se han integrado en hospitales para optimizar la gestión de camas.",
    "La IA permite predecir respuestas a inmunoterapia en pacientes con melanoma.",
    "Me gusta el helado de chocolate.",  # Irrelevant
    "El tiempo está muy agradable hoy.",  # Irrelevant
]

print(f"Query: {query}")
print(f"Documentos: {len(documents)}")

<a name="cross"></a>
## 2. Cross-Encoder

El **Cross-Encoder** procesa query y documento **juntos** a través del modelo, permitiendo atención cruzada entre ellos.

In [ ]:
from sentence_transformers import CrossEncoder

# Load Cross-Encoder model (free, open source)
print("Cargando Cross-Encoder...")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
print("Modelo cargado ✓")

In [ ]:
# Create (query, document) pairs
pairs = [(query, doc) for doc in documents]

# Get relevance scores
scores = cross_encoder.predict(pairs)

# Sort by score
results = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

print("\n🔴 Reranking con Cross-Encoder:")
print("=" * 60)
for i, (doc, score) in enumerate(results, 1):
    emoji = "✅" if score > 0 else "❌"
    print(f"{i}. {emoji} Score: {score:.4f}")
    print(f"   {doc[:70]}...\n")

<a name="bi"></a>
## 3. Bi-Encoder (Dense Retrieval)

El **Bi-Encoder** genera embeddings **separados** para query y documentos, luego calcula similitud.

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load Bi-Encoder model
print("Cargando Bi-Encoder...")
bi_encoder = SentenceTransformer("all-MiniLM-L6-v2")
print("Modelo cargado ✓")

In [ ]:
# Encode query and documents separately
query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
doc_embeddings = bi_encoder.encode(documents, convert_to_tensor=True)

# Calculate similarities
similarities = util.cos_sim(query_embedding, doc_embeddings)[0]

# Sort by similarity
results_bi = sorted(zip(documents, similarities.tolist()), 
                    key=lambda x: x[1], reverse=True)

print("\n🔵 Ranking con Bi-Encoder:")
print("=" * 60)
for i, (doc, score) in enumerate(results_bi, 1):
    emoji = "✅" if score > 0.3 else "❌"
    print(f"{i}. {emoji} Score: {score:.4f}")
    print(f"   {doc[:70]}...\n")

### Comparación: Cross-Encoder vs Bi-Encoder

In [ ]:
import time

# Benchmark speed
n_docs = 100
test_docs = documents * 10  # Create 100 documents

# Bi-Encoder timing
start = time.time()
q_emb = bi_encoder.encode(query, convert_to_tensor=True)
d_embs = bi_encoder.encode(test_docs, convert_to_tensor=True)
sims = util.cos_sim(q_emb, d_embs)
bi_time = time.time() - start

# Cross-Encoder timing
start = time.time()
pairs = [(query, doc) for doc in test_docs]
scores = cross_encoder.predict(pairs)
cross_time = time.time() - start

print("Comparación de velocidad:")
print(f"  Bi-Encoder:    {bi_time:.3f}s para {n_docs} docs")
print(f"  Cross-Encoder: {cross_time:.3f}s para {n_docs} docs")
print(f"  Cross-Encoder es {cross_time/bi_time:.1f}x más lento")

<a name="llm"></a>
## 4. LLM como Reranker

Un LLM puede evaluar la relevancia de documentos con alta precisión, pero es el método más costoso.

In [ ]:
import os
from getpass import getpass

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass("Introduce tu GROQ API Key: ")

from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)
print("LLM configurado ✓")

In [ ]:
def llm_rerank(query: str, documents: list, top_k: int = 5) -> list:
    """Use LLM to rerank documents by relevance."""
    
    # Format documents with indices
    doc_list = "\n".join([f"[{i+1}] {doc}" for i, doc in enumerate(documents)])
    
    prompt = f"""Ordena los siguientes documentos por relevancia para la pregunta.
Devuelve SOLO los números de los {top_k} documentos más relevantes, separados por comas.

Pregunta: {query}

Documentos:
{doc_list}

Top {top_k} más relevantes (solo números, ej: 3,1,5,2,4):"""
    
    response = llm.invoke(prompt)
    
    # Parse response
    try:
        indices = [int(x.strip()) - 1 for x in response.content.split(",")[:top_k]]
        return [(documents[i], i+1) for i in indices if 0 <= i < len(documents)]
    except:
        return [(doc, i) for i, doc in enumerate(documents[:top_k], 1)]

# Test LLM reranking
print("🟣 Reranking con LLM:")
print("=" * 60)

llm_results = llm_rerank(query, documents, top_k=5)
for i, (doc, original_idx) in enumerate(llm_results, 1):
    print(f"{i}. (original #{original_idx}) {doc[:60]}...")

<a name="combinadas"></a>
## 5. Estrategias Combinadas

La mejor práctica es combinar Bi-Encoder (rápido, primera fase) con Cross-Encoder (preciso, segunda fase).

In [ ]:
def two_stage_retrieval(query: str, documents: list, 
                        bi_model, cross_model,
                        first_k: int = 10, final_k: int = 5):
    """
    Two-stage retrieval:
    1. Bi-Encoder for fast initial retrieval
    2. Cross-Encoder for precise reranking
    """
    # Stage 1: Bi-Encoder
    q_emb = bi_model.encode(query, convert_to_tensor=True)
    d_embs = bi_model.encode(documents, convert_to_tensor=True)
    sims = util.cos_sim(q_emb, d_embs)[0]
    
    # Get top-k candidates
    top_indices = sims.argsort(descending=True)[:first_k]
    candidates = [(documents[i], sims[i].item()) for i in top_indices]
    
    print(f"Stage 1 (Bi-Encoder): {len(candidates)} candidatos")
    
    # Stage 2: Cross-Encoder
    pairs = [(query, doc) for doc, _ in candidates]
    scores = cross_model.predict(pairs)
    
    # Final ranking
    final_results = sorted(zip([c[0] for c in candidates], scores), 
                          key=lambda x: x[1], reverse=True)[:final_k]
    
    print(f"Stage 2 (Cross-Encoder): {len(final_results)} resultados finales")
    
    return final_results

# Test two-stage retrieval
print("\n🔶 Two-Stage Retrieval:")
print("=" * 60)

final_results = two_stage_retrieval(
    query, documents,
    bi_encoder, cross_encoder,
    first_k=8, final_k=5
)

print("\nResultados finales:")
for i, (doc, score) in enumerate(final_results, 1):
    print(f"{i}. Score: {score:.4f}")
    print(f"   {doc[:70]}...\n")

<a name="ejercicios"></a>
## 6. Ejercicios Prácticos

### Ejercicio 1: Comparar rankings

In [ ]:
# Exercise 1: Compare all three methods on a new query

new_query = "¿Cómo ayuda la inteligencia artificial a diagnosticar enfermedades?"

# Get rankings from all three methods
# 1. Bi-Encoder
# 2. Cross-Encoder  
# 3. LLM

# Compare which documents appear in top 3 for each
# Are they the same? Different?

print(f"Query: {new_query}")
print("\nCompara los rankings de cada método...")

## Resumen

En este notebook hemos aprendido:

1. **Reranking**: Segunda fase para mejorar precisión
2. **Cross-Encoder**: Preciso pero lento
3. **Bi-Encoder**: Rápido pero menos preciso
4. **LLM Reranker**: Muy preciso pero costoso
5. **Two-stage**: Combinar lo mejor de ambos

### Recomendaciones

| Escenario | Estrategia recomendada |
|-----------|------------------------|
| Latencia crítica | Solo Bi-Encoder |
| Precisión crítica | Two-stage (Bi + Cross) |
| Pocos documentos | Cross-Encoder directo |
| Máxima precisión | LLM Reranker |

En el siguiente notebook veremos **Agentes**, que combinan LLMs con herramientas y razonamiento.

---

## Referencias

- [Sentence Transformers](https://www.sbert.net/)
- [Cross-Encoders for Reranking](https://www.sbert.net/examples/applications/cross-encoder/README.html)